<a href="https://www.kaggle.com/code/thakursankalp/xgboost-arima-lstm-store-sales-forecasting?scriptVersionId=131949099" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# The notebook is structured as follows:

**Import Libraries**: This section imports necessary Python libraries and modules for data manipulation, visualization, and machine learning.

**Load Datasets**: Five different datasets related to store sales, store information, oil prices, holiday events, and transactions are loaded in this step.

**Preprocessing**: This step transforms raw data into a more digestible format for machine learning algorithms. We perform a wide range of preprocessing tasks such as converting date columns to datetime objects, encoding categorical features, merging data from different datasets, handling missing values, and generating new features.

**Model Building & Evaluation**: Here, we train four different models: XGBoost, ARIMA, LSTM, and Prophet. Each model is chosen for its specific strengths in handling time-series data. After training, each model's performance is evaluated using the Mean Squared Error (MSE) metric.

**Submission**: Based on the performance of the models, we choose the best one and use it to make predictions on the test set. Finally, we prepare a CSV file for submission, which includes predicted sales for given dates.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [2]:
# Load the datasets
train_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
stores_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
transactions_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

# Preprocessing


In [3]:
# Convert date columns to datetime objects
train_df['date'] = pd.to_datetime(train_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'])

# Merge train data with store, oil, and transactions data
train_df = train_df.merge(stores_df, on='store_nbr', how='left')
train_df = train_df.merge(oil_df, on='date', how='left')
train_df = train_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')

# Encode categorical features
le = LabelEncoder()
train_df['city'] = le.fit_transform(train_df['city'])
train_df['state'] = le.fit_transform(train_df['state'])
train_df['type'] = le.fit_transform(train_df['type'])

# Create holiday features
holidays_df['holiday'] = 1
train_df = train_df.merge(holidays_df[['date', 'holiday']], on='date', how='left')
train_df['holiday'].fillna(0, inplace=True)

# One-hot encode the 'family' column
ohe_family = OneHotEncoder(sparse=False, dtype=int)
family_encoded = ohe_family.fit_transform(train_df[['family']])
family_columns = ohe_family.get_feature_names_out(['family'])
family_df = pd.DataFrame(family_encoded, columns=family_columns, index=train_df.index)
train_df = pd.concat([train_df, family_df], axis=1)

# Drop the original 'family' column
train_df.drop('family', axis=1, inplace=True)

# Create lag features for oil prices
lags = [1, 7, 14]
for lag in lags:
    train_df[f'oil_lag_{lag}'] = train_df['dcoilwtico'].shift(lag)

# Create a feature for payday (every 15th and last day of the month)
train_df['payday'] = ((train_df['date'].dt.day == 15) | (train_df['date'].dt.is_month_end)).astype(int)

# Add date, month, year as features
train_df['day'] = train_df['date'].dt.day
train_df['month'] = train_df['date'].dt.month
train_df['year'] = train_df['date'].dt.year

# Add weekdays as one hot encoded features
train_df['weekday'] = train_df['date'].dt.day_name()
ohe = OneHotEncoder(sparse=False, dtype=int)
weekday_encoded = ohe.fit_transform(train_df[['weekday']])
weekday_columns = ohe.get_feature_names_out(['weekday'])
weekday_df = pd.DataFrame(weekday_encoded, columns=weekday_columns, index=train_df.index)
train_df = pd.concat([train_df, weekday_df], axis=1)

# Drop the original 'family' column
train_df.drop('weekday', axis=1, inplace=True)


# Drop rows with missing values
train_df.dropna(inplace=True)

# Add polynomial features
poly_features = ['day', 'month', 'year', 'oil_lag_1', 'oil_lag_7', 'oil_lag_14']
poly = PolynomialFeatures(degree=2, include_bias=False)
train_df_poly = poly.fit_transform(train_df[poly_features])


In [4]:
train_df.head()

,id,date,store_nbr,sales,onpromotion,city,state,type,cluster,dcoilwtico,...,day,month,year,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
1796,1796,2013-01-02,1,3.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1797,1797,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1798,1798,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1799,1799,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1800,1800,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1


# Train Test Split

In [5]:
# Split the data into train and test sets
train_data, test_data = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=True)

# Define the features and target variable
X_train = train_data.drop(['date', 'store_nbr', 'sales', 'dcoilwtico'], axis=1)
y_train = train_data['sales']
X_test = test_data.drop(['date', 'store_nbr', 'sales', 'dcoilwtico'], axis=1)
y_test = test_data['sales']

# Scale the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=0.95)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# **XGBoost** & **ARIMA** Compared


In [7]:
from statsmodels.tsa.arima.model import ARIMA

# Define the XGBoost model
model_xg = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    min_child_weight=1,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='gpu_hist' 
)

# Define the ARIMA model
model_ar = ARIMA(y_train, order=(2, 0, 0))

# Fit the models
model_xg.fit(X_train, y_train)
model_ar = model_ar.fit()  

# Make predictions
predictions_xg = model_xg.predict(X_test)
predictions_ar = model_ar.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)  

# Calculate the mean squared error
mse_xg = mean_squared_error(y_test, predictions_xg)
mse_ar = mean_squared_error(y_test, predictions_ar)

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


In [8]:
print(mse_xg)
print(mse_ar)

102014.21365980568
1011636.7909510805


# LSTM Results

LSTM is a type of Recurrent Neural Network (RNN) that is very effective in predicting time series data because it can remember patterns over the time period.

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Reshape input to be 3D [samples, timesteps, features]
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [10]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [11]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='tanh', recurrent_activation='sigmoid', recurrent_dropout=0, unroll=False, use_bias=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')

history = model_lstm.fit(X_train_lstm, y_train, epochs=5, batch_size=72,
                         validation_data=(X_test_lstm, y_test), verbose=2, shuffle=False)

# make a prediction
predictions_lstm = model_lstm.predict(X_test_lstm)

# flatten predictions
predictions_lstm = predictions_lstm.flatten()

# calculate the mean squared error
mse_lstm = mean_squared_error(y_test, predictions_lstm)


Epoch 1/5
21443/21443 - 107s - loss: 275.8298 - val_loss: 236.7364 - 107s/epoch - 5ms/step
Epoch 2/5
21443/21443 - 100s - loss: 216.7253 - val_loss: 200.8682 - 100s/epoch - 5ms/step
Epoch 3/5
21443/21443 - 92s - loss: 189.5122 - val_loss: 179.9238 - 92s/epoch - 4ms/step
Epoch 4/5
21443/21443 - 100s - loss: 172.6794 - val_loss: 166.2901 - 100s/epoch - 5ms/step
Epoch 5/5
21443/21443 - 91s - loss: 161.4654 - val_loss: 156.9706 - 91s/epoch - 4ms/step
12062/12062 [==============================] - 25s 2ms/step


In [12]:
print(mse_lstm)

463960.64486114826


# Prophet results

Prophet is an open-source library developed by Facebook that is specifically designed for forecasting time series data. It's robust to missing data, shifts in the trend, and large outliers.



# Submission

In [13]:
scaler = MinMaxScaler()
scaler.fit(y_train.values.reshape(-1, 1))

MinMaxScaler()

In [14]:
# Choose the model with the lowest mean squared error
if min(mse_xg, mse_ar, mse_lstm) == mse_xg:
    model = model_xg
elif min(mse_xg, mse_ar, mse_lstm) == mse_ar:
    model = model_ar
else:
    model = model_lstm

# Make predictions using the chosen model
if model == model_lstm:
    X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    predictions = model.predict(X_test_lstm)
else:
    predictions = model.predict(X_test)

# Inverse transform the predictions
predictions = predictions.reshape(-1, 1)
predictions = scaler.inverse_transform(predictions)

# Create a submission DataFrame
submission = pd.DataFrame({"sales": predictions.flatten()})
# Assume that 'id' values are a range from 1 to the length of predictions
submission = pd.DataFrame({"id": range(1, len(predictions) + 1), "sales": predictions.flatten()})

# Save the submission DataFrame as a CSV file
submission.to_csv("submission.csv", index=False)


In [15]:
predictions

array([[ 2169509.8],
       [16019837. ],
       [-4425233. ],
       ...,
       [ 5335895. ],
       [17299202. ],
       [-1551424. ]], dtype=float32)

In [16]:
submission.head(-10)

,id,sales
0,1,2.169510e+06
1,2,1.601984e+07
2,3,-4.425233e+06
3,4,2.448682e+06
4,5,3.385162e+07
...,...,...
385949,385950,1.279328e+06
385950,385951,1.143348e+08
385951,385952,-2.462559e+06
385952,385953,9.373693e+07
